In [223]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [224]:
train = pd.read_csv("./wind/train.csv")
test = pd.read_csv("./wind/test.csv")
submission = pd.read_csv("./wind/sample_submission.csv")

In [225]:
train.drop(['id'], axis=1, inplace=True)

In [226]:
from sklearn.preprocessing import OneHotEncoder

In [227]:
x.columns

Index(['temperature', 'pressure', 'humidity', 'wind_speed', 'wind_direction',
       'precipitation', 'snowing', 'cloudiness'],
      dtype='object')

In [228]:
test.shape

(19275, 9)

In [229]:
x = train.drop(['target'], axis = 1)
y = train['target']

In [271]:
from sklearn.model_selection import train_test_split

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=1, random_state=42) 

In [272]:
x_train.shape, x_test.shape, y_train.shape, y_test.shape

((19274, 8), (1, 8), (19274,), (1,))

In [255]:
enc_cols = ['snowing']
normal_cols = list(set(x_train.columns) - set(enc_cols))
normal_cols

['wind_direction',
 'temperature',
 'wind_speed',
 'precipitation',
 'humidity',
 'cloudiness',
 'pressure']

In [233]:
enc = OneHotEncoder()

In [241]:
tmp_tr = pd.DataFrame(
    enc.fit_transform(x_train[enc_cols]).toarray(), #변환
    columns = enc.get_feature_names_out() 
)
enc_tr = pd.concat(
    [x_train[normal_cols].reset_index(drop=True), tmp_tr.reset_index(drop=True)]
    , axis=1
)
# test
tmp_te = pd.DataFrame(
    enc.transform(x_test[enc_cols]).toarray(), 
    columns = enc.get_feature_names_out()
)
enc_te = pd.concat(
    [x_test[normal_cols].reset_index(drop=True), tmp_te.reset_index(drop=True)]
    , axis=1
)
enc_tr.head()

IndexError: only integers, slices (`:`), ellipsis (`...`), numpy.newaxis (`None`) and integer or boolean arrays are valid indices

In [235]:
tmp_t_te = pd.DataFrame(
    enc.transform(test[enc_cols]).toarray(), 
    columns = enc.get_feature_names_out()
)
enc_t_te = pd.concat(
    [test[normal_cols].reset_index(drop=True), tmp_t_te.reset_index(drop=True)]
    , axis=1
)

NotFittedError: This OneHotEncoder instance is not fitted yet. Call 'fit' with appropriate arguments before using this estimator.

In [ ]:
enc_t_te.head()

In [ ]:
enc_te.head(20)

In [ ]:
enc_tr = enc_tr

In [256]:
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import StandardScaler, RobustScaler
from sklearn.model_selection import GridSearchCV, cross_val_score
from sklearn.metrics import accuracy_score
import lightgbm as lgb

sc = StandardScaler()
rc = RobustScaler()

In [257]:
x_train = sc.fit_transform(enc_tr)
x_test = sc.fit_transform(enc_te)

In [258]:
x_train1 = rc.fit_transform(enc_tr)
x_test1 = rc.fit_transform(enc_te)

In [259]:
modelV0 = DecisionTreeRegressor(max_depth=3, random_state=42)
modelV0.fit(x_train, y_train)
modelV0.score(x_train, y_train), modelV0.score(x_test1, y_test)

(0.14038321169763723, 0.09935916542615353)

In [260]:
modelV1 = RandomForestRegressor(n_estimators = 100, random_state=42)
modelV1.fit(x_train1, y_train)
modelV1.score(x_train1, y_train), modelV1.score(x_test1, y_test)

(0.905890997449412, 0.2937785704640338)

In [ ]:
!pip install shap

In [261]:
from xgboost import XGBClassifier,plot_importance
hp = {
    "random_state" : 42,
    "max_depth" : 2,
    "n_estimators" : 200, # 수행할 부스팅 단계 수
    "learning_rate": 0.01,
    "min_child_weight": 2,
    "gamma": 1
}

In [262]:
xgb = XGBClassifier(**hp).fit(x_train, y_train)
print(f'훈련용 평가지표: {xgb.score(x_train, y_train)} / 테스트용 평가지표: {xgb.score(x_test, y_test)}')

ValueError: Invalid classes inferred from unique values of `y`.  Expected: [   0    1    2 ... 8127 8128 8129], got [ 0.     0.243  0.26  ... 17.125 17.256 17.344]

In [264]:
from lightgbm import LGBMClassifier, plot_importance

In [265]:
hp = {
    "random_state" : 42,
    "max_depth" : 2,
    "n_estimators" : 100,
    "learning_rate": 0.01,
}

lgb = LGBMClassifier(**hp).fit(x_train, y_train)
print(f'훈련용 평가지표: {lgb.score(x_train, y_train)} / 테스트용 평가지표: {lgb.score(x_test, y_test)}')

ValueError: Unknown label type: 'continuous'

In [238]:
submission = pd.read_csv('./wind/sample_submission.csv')
submission.head()

,id,target
0,TEST_00000,7.33243
1,TEST_00001,7.92765
2,TEST_00002,7.96670
3,TEST_00003,7.27823
4,TEST_00004,7.27823


In [239]:
ids = test["id"]
predictions = modelV1.predict(enc_t_te)

submission = pd.DataFrame({'id' : ids, 'target': predictions})
submission.to_csv('./wind/sample_submission.csv', index=False)

C:\Users\Playdata\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\base.py:413: UserWarning: X has feature names, but RandomForestRegressor was fitted without feature names
  warnings.warn(
